In [157]:
import h5py
import numba
import numpy as np
import pandas as pd
import scipy.integrate
import matplotlib.pyplot as plt
from hoki.utils.exceptions import HokiFormatError

import numba as nb
import numpy as np


In [158]:
time_bins = np.arange(6.0, 11.1, 0.1)
linear_time_edges = np.append([0.0], 10**np.arange(6.05, 11.15, 0.1))
linear_time_intervals = np.diff(linear_time_edges)
metallicity_names = ["zem5", "zem4", "z001","z002", "z003", "z004", "z006", "z008", "z010", "z014", "z020", "z030", "z040"]
metallicity_values = np.array([0.00001, 0.0001, 0.001, 0.002, 0.003, 0.004, 0.006, 0.008, 0.010, 0.014, 0.020, 0.030, 0.040])
event_types = ["Ia", "IIP", "II", "Ib", "Ic", "LGRB", "PISNe", "low_mass"]
all_names = ['log_age', 'Ia', 'IIP', 'II', 'Ib', 'Ic', 'LGRB', 'PISNe', 'low_mass', 'e_Ia', 'e_IIP', 'e_II', 'e_Ib', 'e_Ic', 'e_LGRB', 
             'e_PISNe', 'e_low_mass', 'age_yrs']
current_time = 13.799e9

In [159]:
@nb.njit(parallel=True, cache=True)
def _grid_rate_calculator_over_time(bpass_rate, SFH, time_points, nr_time_bins):
    event_rates = np.empty(nr_time_bins, dtype=np.float64)
    time_edges = np.linspace(0, current_time, nr_time_bins + 1)
    mass_per_bin = _optimised_mass_per_bin(time_points, SFH, time_edges, sample_rate=25)

    for count in nb.prange(nr_time_bins):
        p1 = time_edges[count]
        p2 = time_edges[count + 1]
        event_rate = _integral(
            p2, p1, time_edges, bpass_rate, linear_time_intervals
        )
        event_rates[count] = event_rate / (p2 - p1)

    return event_rates




def _over_time(mass_per_bin, time_edges, bpass_rate):
    """
    Calculates the events rates per bin over the given bin edges.

    Parameters
    ----------
    mass_per_bin (ndarray) : shape N - array containing the mass per time bin
    time_edges (ndarray) : shape N+1 - bin edges of mass_per_bin
    bpass_rate (ndarray) : array of shape (51,) - BPASS event rates at a specific metallicity and time bins

    Returns
    -------
    event_rate (ndarray) : shape N - number of events per bin
    """
    event_rate = np.zeros(len(mass_per_bin))

    for count in range(len(mass_per_bin)):
        t = time_edges[count + 1]
        for num in range(0, count + 1):
            p1 = t - time_edges[num]
            p2 = t - time_edges[num + 1]
            bin_events = _integral(p2, p1, linear_time_edges, bpass_rate, linear_time_intervals)
            event_rate[num] += bin_events * mass_per_bin[count]
    return event_rate


class CSPEventRate:
    def __init__(self, bpass_rate):
        self.bpass_rate = bpass_rate / (1e6 * linear_time_intervals)

    def grid_over_time(self, SFH_list, time_points, nr_time_bins, return_time_edges=False):
        nr_sfh = SFH_list.shape[0]
        time_edges = np.linspace(0, current_time, nr_time_bins + 1)
        event_rate_list = np.zeros((nr_sfh, nr_time_bins), dtype=np.float64)

        for sfh in range(nr_sfh):
            # Pass self.bpass_rate as an argument
            event_rate_list[sfh] = _grid_rate_calculator_over_time(
                self.bpass_rate, SFH_list[sfh], time_points, nr_time_bins
            )

        if return_time_edges:
            return np.array([event_rate_list, time_edges], dtype=object)
        else:
            return event_rate_list


In [160]:
import numba as nb

@nb.njit(parallel=True, cache=True)
def linear_interp(x, xp, fp):
    # Function to perform linear interpolation using explicit indexing
    result = np.empty_like(x)
    for i in nb.prange(len(x)):
        j = np.searchsorted(xp, x[i])
        j = max(1, min(j, len(xp) - 1))
        x0, x1 = xp[j - 1], xp[j]
        y0, y1 = fp[j - 1], fp[j]
        result[i] = y0 + (y1 - y0) * (x[i] - x0) / (x1 - x0)
    return result




@nb.njit(parallel=True, cache=True)
def _optimised_mass_per_bin(time_points, sfh, time_edges, sample_rate=25):
    nr_bins = len(time_edges) - 1
    out = np.empty(nr_bins - 1)

    for num in range(nr_bins - 1):
        s = 0
        x = np.linspace(time_edges[num], time_edges[num + 1], sample_rate)
        y = linear_interp(x, time_points, sfh)

        for N in range(1, len(x)):
            s += (x[N] - x[N - 1]) * (y[N] + y[N - 1])

        out[num] = s / 2

    return out


In [161]:
@staticmethod
def _get_bin_index(x, edges):
    """
    Gets the bin number given the edges.

    Parameters
    ----------
    x (float) : value where the bin number is needed to be known
    edges (ndarray) : array with the edges of the histogram

    Returns
    -------
    out (int) : bin index of x

    """
    if x < edges[0] or x > edges[-1]:  # checks if x is within the edge ranges
        raise ValueError("x outside of range")

    out = 0
    if x == edges[-1]:  # if x is equal to the last bin, return len-2
        out = len(edges) - 2

    for i, val in enumerate(edges):
        if val > x:  # x larger than bin, return answer
            out = i - 1
            break
    return int(out)

In [162]:
@nb.njit
def _integral(x1, x2, edges, values, bin_width):
    lower_bin = _get_bin_index(x1, edges)
    upper_bin = _get_bin_index(x2, edges)
    total = 0.0

    for i in range(lower_bin, upper_bin + 1):
        if i == lower_bin:
            lower = x1
        else:
            lower = edges[i]

        if i == upper_bin:
            upper = x2
        else:
            upper = edges[i + 1]

        total += (upper - lower) * values[i]

    return total


In [163]:

''' 
@staticmethod
@nb.njit(parallel=True, cache=True)

def _over_time(mass_per_bin, time_edges, bpass_rate):
    """
    Calculates the events rates per bin over the given bin edges.

    Parameters
    ----------
    mass_per_bin (ndarray) : shape N - array containing the mass per time bin
    time_edges (ndarray) : shape N+1 - bin edges of mass_per_bin
    bpass_rate (ndarray) : array of shape (51,) - BPASS event rates at a specific metallicity and time bins

    Returns
    -------
    event_rate (ndarray) : shape N - number of events per bin
    """
    event_rate = np.zeros(len(mass_per_bin))

    for count in range(len(mass_per_bin)):
        t = time_edges[count + 1]
        for num in range(0, count + 1):
            p1 = t - time_edges[num]
            p2 = t - time_edges[num + 1]
            bin_events = _integral(p2, p1, linear_time_edges, bpass_rate, linear_time_intervals)
            event_rate[num] += bin_events * mass_per_bin[count]
    return event_rate

'''''' 

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-163-58f227ae8b24>, line 30)

In [164]:
file = h5py.File("/Users/dillon/Desktop/data.h5", "r")
SFH_1 = file['SFH']['TNG']


@np.vectorize
def calc_LB(z, omega0, omega1, hubble):
    '''
    Calculates the lookback time according to Hobbs.

    Parameters:
    z : float
        The redshift at which you want to calculate the lookback
    omega0 : float
        The Matter density parameter (0.3111 Planck 2018)
    omega1 : float
        The dark energy density parameter (0.6889 Planck 2018)
    hubble : float
        The hubble parameters (0.6766 Planck 2018)
    '''
    def func(x):
        E = np.sqrt(omega0*(1+x)**3 + omega1)
        return 1/((1+x)*E)
    # can be simplified by already doing the extra coefficient calculations
    t_hubble = (1/(100*hubble))*3.0856776*10**19
    return t_hubble *scipy.integrate.quad(func, 0, z)[0]/(60*60*24*365.2388526)

lookback_time = calc_LB(SFH_1.attrs['redshift'], 0.3111, 0.6889, 0.6766)

In [165]:
lookback_time 

array([3.20898781e-06, 1.36651743e+08, 3.40473695e+08, 4.75571226e+08,
       6.77019125e+08, 8.10505604e+08, 1.00949470e+09, 1.14131404e+09,
       1.33776091e+09, 1.46785804e+09, 1.66168105e+09, 1.85389144e+09,
       1.98112192e+09, 2.17058255e+09, 2.29595457e+09, 2.48259040e+09,
       2.66749669e+09, 2.78979430e+09, 2.97176279e+09, 3.15193684e+09,
       3.27104472e+09, 3.50681178e+09, 3.62345859e+09, 3.79686627e+09,
       3.96838213e+09, 4.08166616e+09, 4.24998988e+09, 4.41637672e+09,
       4.58081193e+09, 4.74328198e+09, 4.90377458e+09, 5.06227866e+09,
       5.21878445e+09, 5.37328341e+09, 5.52576832e+09, 5.72593842e+09,
       5.87370287e+09, 6.01943833e+09, 6.16314307e+09, 6.35158982e+09,
       6.49055642e+09, 6.67269173e+09, 6.80693293e+09, 6.98278374e+09,
       7.11232701e+09, 7.28193939e+09, 7.44801459e+09, 7.61057668e+09,
       7.73020893e+09, 7.92527255e+09, 8.07746875e+09, 8.22627635e+09,
       8.37173270e+09, 8.51387729e+09, 8.68696424e+09, 8.82181176e+09,
      

In [166]:
path = "/Users/dillon/Desktop"
data = h5py.File(f"{path}/data.h5", "r")


DTD = data["DTD"]

event_types = ["Ia", "IIP", "II", "Ib", "Ic", "LGRB", "PISNe", "BBH", "BHNS", "BNS"]
metallicities = ["z001", "z002", "z003", "z004", "z006", "z008", "z010", "z014", "z020", "z030", "z040", "zem4", "zem5" ]


event_total = {}
for i in (metallicities):
    event_total[i] = {}
    for j in event_types:
        event_total[i][j] = DTD[j][i][:]
    event_total[i]["CCSN"] = event_total[i]["II"] + event_total[i]["Ib"] + event_total[i]["Ic"] + event_total[i]["IIP"]


Z12 = metallicities[12] 

x1 = event_total[Z12]
DTD = x1["Ia"]


In [167]:
bpass_rate = DTD

In [168]:
er_csp = CSPEventRate(bpass_rate)

In [169]:
er_csp

In [170]:
single_SFH = np.array([8.33191811e-03, 8.88059988e-03, 9.19159485e-03, 9.34190314e-03,
       9.52613027e-03, 9.61924736e-03, 9.95608132e-03, 1.01121330e-02,
       1.04936439e-02, 1.07714432e-02, 1.11501214e-02, 1.15528803e-02,
       1.16737580e-02, 1.21275267e-02, 1.23668919e-02, 1.25800074e-02,
       1.29328367e-02, 1.30097050e-02, 1.33423500e-02, 1.38473763e-02,
       1.41944978e-02, 1.45959858e-02, 1.50736653e-02, 1.55658660e-02,
       1.60191941e-02, 1.66433717e-02, 1.68851090e-02, 1.77012000e-02,
       1.80640072e-02, 1.86711321e-02, 1.91592408e-02, 1.99370643e-02,
       2.03300593e-02, 2.10643742e-02, 2.16123611e-02, 2.24611632e-02,
       2.27264639e-02, 2.39028158e-02, 2.46715799e-02, 2.54031622e-02,
       2.64623204e-02, 2.69635787e-02, 2.78400452e-02, 2.95416449e-02,
       3.03482792e-02, 3.11052665e-02, 3.19822080e-02, 3.27873986e-02,
       3.34144609e-02, 3.43084579e-02, 3.54056765e-02, 3.60838504e-02,
       3.78456969e-02, 3.85507297e-02, 4.08587507e-02, 4.24272177e-02,
       4.36311638e-02, 4.50121161e-02, 4.62638004e-02, 4.69334210e-02,
       4.82076153e-02, 4.97647258e-02, 4.99519472e-02, 5.16775608e-02,
       5.31130534e-02, 5.30846088e-02, 5.46388458e-02, 5.67281525e-02,
       5.86522957e-02, 5.90331215e-02, 6.14186184e-02, 6.25037283e-02,
       6.56813424e-02, 6.29126696e-02, 6.17826163e-02, 5.98024055e-02,
       5.51713339e-02, 5.19833105e-02, 4.57502257e-02, 4.25872869e-02,
       3.81978491e-02, 3.23877045e-02, 2.64170327e-02, 2.34498785e-02,
       1.98288241e-02, 1.55873013e-02, 1.39437820e-02, 9.54679524e-03,
       6.47984793e-03, 5.44900603e-03, 4.16569436e-03, 2.95379814e-03,
       2.11928454e-03, 1.40211279e-03, 1.04131959e-03, 6.62699202e-04,
       3.25848035e-04, 1.49320079e-04, 8.09204735e-06, 0.00000000e+00])

In [171]:
out = er_csp.grid_over_time(single_SFH, lookback_time, 100)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(float64, int64)
 
There are 16 candidate implementations:
[1m      - Of which 16 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(float64, int64)':[0m
[1m       No match.[0m
[0m
[0m[1mDuring: typing of intrinsic-call at <ipython-input-160-749fc07ed2d5> (11)[0m
[1m
File "<ipython-input-160-749fc07ed2d5>", line 11:[0m
[1mdef linear_interp(x, xp, fp):
    <source elided>
        x0, x1 = xp[j - 1], xp[j]
[1m        y0, y1 = fp[j - 1], fp[j]
[0m        [1m^[0m[0m

[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function linear_interp at 0x7fbc5a082160>))[0m
[0m[1mDur